# Меняем все предложения по заданию

In [ ]:
sentences = ["Распространены крупнолистные миртовые и орхидеи.",
             "До появления на островах человека из млекопитающих на суше обитали летучие мыши, а в прибрежных водах — дельфины.",
             "Таким образом, люди с белой кожей среди темнокожих оказываются жертвой всеобщей жажды потустороннего."]

# Скачиваем данные и библиотеки

In [ ]:
! pip install -q conllu
! pip install -q graphviz
! pip install -q ufal.udpipe
!wget -q 'https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe'
!wget -q 'https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-taiga-ud-2.5-191206.udpipe'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 937.0/937.0 kB 28.9 MB/s eta 0:00:00


# Запускаем код

In [ ]:
import sys

from conllu import parse
from graphviz import Digraph
from ufal.udpipe import Model, Pipeline, ProcessingError
from conllu import parse
from graphviz import Digraph

INPUT_FORMAT = "tokenize"
OUTPUT_FORMAT = "conllu"
SYNTAGRUS_MODEL_FILE = "russian-syntagrus-ud-2.5-191206.udpipe"
TAIGA_MODEL_FILE = "russian-taiga-ud-2.5-191206.udpipe"

syntagrus_model = Model.load(SYNTAGRUS_MODEL_FILE)
taiga_model = Model.load(TAIGA_MODEL_FILE)

syntagrus_pipeline = Pipeline(syntagrus_model, INPUT_FORMAT, Pipeline.DEFAULT, Pipeline.DEFAULT, OUTPUT_FORMAT)
taiga_pipeline = Pipeline(taiga_model, INPUT_FORMAT, Pipeline.DEFAULT, Pipeline.DEFAULT, OUTPUT_FORMAT)

error = ProcessingError()

def parse_syntax(raw_text, pipeline):
    text = raw_text.strip()
    processed = pipeline.process(text, error)

    if error.occurred():
        sys.stderr.write("An error occurred when running run_udpipe: ")
        sys.stderr.write(error.message)
        sys.stderr.write("\n")

    return processed

def parse_output(conll):
    return [s.to_tree() for s in parse(conll)]

class Root(object):
  def __init__(self, child):
    self.token = {"form": "", "id": 0, "deprel": "root"}
    self.children = [child]

def edges_sets(tree):
  edges_labeled = []
  edges_unlabeled = []
  queue = [Root(tree)]
  
  while len(queue) > 0:
    head = queue.pop()
    head_label = head.token["form"] + " (%d)" % head.token["id"]
    queue.extend(head.children)
    
    for c in head.children:
      c_label = c.token["form"] + " (%d)" %  c.token["id"]
      edges_unlabeled.append((head_label, c_label))
      edges_labeled.append((head_label, c_label, c.token["deprel"]))
  
  return set(edges_labeled), set(edges_unlabeled)

def accuracy(gold, pred):
  gold_labeled, gold_unlabeled = edges_sets(gold)
  pred_labeled, pred_unlabeled = edges_sets(pred)

  return len(gold_labeled.intersection(pred_labeled)) / len(gold_labeled), \
          len(gold_unlabeled.intersection(pred_unlabeled)) / len(gold_unlabeled)

print("UAS \tLAS \tMAX(Глубина) \tПредложение")
for sentence in sentences:
  syntag = parse_syntax(sentence, syntagrus_pipeline)
  taiga = parse_syntax(sentence, taiga_pipeline)

  str_tree = parse_output(syntag.strip())[0]
  taiga_tree = parse_output(taiga.strip())[0]

  LAS, UAS = accuracy(str_tree, taiga_tree)

  queues = [[(taiga_tree, 0)], [(str_tree, 0)]]
  new_depth = []

  for queue in queues:
    depth = 0
    while len(queue) > 0:
        head, depth = queue.pop(0)
        queue.extend([(c, depth + 1) for c in head.children])
    new_depth.append(depth)
  
  
  print("%.3f \t%.3f \t%d \t\t%s" % (UAS, LAS, max(new_depth), sentence))

UAS 	LAS 	MAX(Глубина) 	Предложение
0.500 	0.500 	3 		Распространены крупнолистные миртовые и орхидеи.
0.800 	0.750 	3 		До появления на островах человека из млекопитающих на суше обитали летучие мыши, а в прибрежных водах — дельфины.
0.800 	0.600 	4 		Таким образом, люди с белой кожей среди темнокожих оказываются жертвой всеобщей жажды потустороннего.
